<a href="https://colab.research.google.com/github/serikkk84/practicum/blob/main/identification%20of%20comments/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%82%D0%BE%D0%BA%D1%81%D0%B8%D1%87%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель со значением метрики качества F1 не меньше 0.75 для классификации комментариев на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

### Загрузка данных

Загрузим необходимые для исследования библиотеки

In [ ]:
import numpy as np
import pandas as pd
import re 
import time
import nltk
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords as nltk_stopwords


Загрузим данные и посмотрим

In [ ]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')
data

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


Почти 160 тысяч комментариев. Очень много...

Посмотрим состав данных и вероятные пропуски

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Все логично, пропусков нет

Посмотрим на соотношение позитивных и негативных отзывов

In [ ]:
negative_posts = data.query('toxic == 1').count()/len(data)
print('Процентное соотношение негативных отзывов составляет', negative_posts['toxic'])
print('Позитивных отзывов', 1 - negative_posts['toxic'])

Процентное соотношение негативных отзывов составляет 0.10161213369158527
Позитивных отзывов 0.8983878663084147


Почти 90 процентов позитивных отзывов. Возможен дисбаланс классов. Пока бороться с дисбалансом не буду, посмотрю на получаемый результат.

### Подготовка текстовых данных

Преобразуем комментарии, оставив только буквы в заданых регистрах и цифры

In [ ]:
text_new = []
pattern = r'[^a-zA-Z0-9]' 
for sentence in data.text:
  cleared_text = re.sub(pattern, " ", sentence)
  text_new.append(" ". join(cleared_text.split()))

Добавим столбец с обработанными коментариями в датафрейм

In [ ]:
data['clear_text'] = text_new
data

,Unnamed: 0,text,toxic,clear_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...
159290,159449,And it looks like it was actually you who put ...,0,And it looks like it was actually you who put ...


Можно переходить к обучению

## Обучение

Разделим датафрейм на обучающую и тестовую выборки

Так как объем данных достаточно большой, думаю можно выделить 50% для тестовой выборки. Это позволит сократить время обучения и пресказания

In [ ]:
data_train_common, data_test = train_test_split(data, test_size=0.5, random_state=12345)

Теперь валидационная

In [ ]:
data_train, data_valid = train_test_split(data_train_common, test_size=0.2, random_state=12345)

### Логистическая регрессия

Перед началом обучения, напишем две функции ля подсчета времени обучения и предсказания. По заданию необязательно, тем не менее будет интересно посмотреть, учитывая такой большой объем данных

In [ ]:
def learn_time(model, x_train, y_train):
    start_time = time.time()
    model.fit(x_train,y_train)
    finish_time = time.time()
    time_fit = finish_time - start_time
    return time_fit

In [ ]:
def predict_time(model, x_valid):
    start_time = time.time()
    predict = model.predict(x_valid)    
    finish_time = time.time()
    time_predict = finish_time - start_time
    return time_predict

Начнем с логистической регрессии

In [ ]:
corpus = data_train['clear_text'].values #.astype('U')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf = count_tf_idf.fit_transform(data_train)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Обязательно укажем балансирование классов в параметрах

In [ ]:
target_train = data_train['toxic']
features_train = data_train['clear_text']

target_valid = data_valid['toxic']
features_valid = data_valid['clear_text']

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train) 
tf_idf_valid = count_tf_idf.transform(features_valid) 
tf_idf_train.shape
model = LogisticRegression(C=10.0, random_state = 0, solver = 'liblinear', class_weight= 'balanced')
model.fit (tf_idf_train,target_train)
print('F1 метрика для логистической регрессии', f1_score(target_valid, model.predict(tf_idf_valid)))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


F1 метрика для логистической регрессии 0.7665965755482126


In [ ]:
learn_time(model, tf_idf_train,target_train)

18.679446935653687

In [ ]:
predict_time(model, tf_idf_valid)

0.002184152603149414

И сразу есть необходимый результат. Приятно, но попробуем улучшить

### Логистическая регрессия Pipeline

Где-то подсмотрел обучение с pipeline. Попробую.

In [ ]:
pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), (
    "clf", LogisticRegression(C=10.0, class_weight= 'balanced', random_state = 0,solver = 'liblinear'))])
pipe.fit(features_train,target_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=10.0, class_weight='balanced',
                                    random_state=0, solver='liblinear'))])

In [ ]:
f1_score(target_valid, pipe.predict(features_valid))

0.7588235294117648

Результат немного хуже

Время 

In [ ]:
learn_time(pipe, features_train, target_train)

28.44417905807495

In [ ]:
predict_time(pipe, features_valid)

0.7134711742401123

### Борьба с дисбалансом

Воспользуемся апсемплином

In [ ]:
from sklearn.utils import shuffle
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

### Логистическая регрессия с отбалансированными классами

Попробуем переобучить успешную модель 

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_upsampled) 
tf_idf_valid = count_tf_idf.transform(features_valid) 
tf_idf_train.shape
model_upsemp = LogisticRegression(C=10.0, random_state = 0, solver = 'liblinear', class_weight= 'balanced')
model_upsemp.fit (tf_idf_train,target_upsampled)
print('F1 метрика для логистической регрессии', f1_score(target_valid, model_upsemp.predict(tf_idf_valid)))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


F1 метрика для логистической регрессии 0.7541882424611636


Показатель метрики ухудшился

Время

In [ ]:
learn_time(model_upsemp, tf_idf_train, target_upsampled)

22.18457555770874

In [ ]:
predict_time(model_upsemp, tf_idf_valid)

0.003007173538208008

### Случайный лес

Вряди эта модель справиться, но попробую

In [ ]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_ran_for = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model_ran_for.fit(tf_idf_train,target_upsampled) 
        predictions_valid = model_ran_for.predict(tf_idf_valid) 
        result = f1_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("F1 наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

F1 наилучшей модели на валидационной выборке: 0.25986234681886855 Количество деревьев: 30 Максимальная глубина: 10


Очень низкий показатель

Время

In [ ]:
learn_time(model_ran_for, tf_idf_train, target_upsampled)

1.6501944065093994

In [ ]:
predict_time(model_ran_for, tf_idf_valid)

0.15833210945129395

### Decision Tree

In [ ]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 50):
    model_dec_tr = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dec_tr.fit(tf_idf_train,target_upsampled) 
    predictions_valid = model_dec_tr.predict(tf_idf_valid) 
    result =  f1_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model_dec_tr
        best_result = result
        best_depth = depth

print("F1 наилучшей модели на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

F1 наилучшей модели на валидационной выборке: 0.6037959667852906 Глубина дерева: 47


Показатель меры ниже необходимого

Время

In [ ]:
learn_time(model_dec_tr, tf_idf_train, target_upsampled)

27.833019971847534

In [ ]:
predict_time(model_dec_tr, tf_idf_valid)

0.006022214889526367

## Выбор лучшей модели и проверка

Объединим данные в таблицу

In [ ]:
models = ['LogisticRegression', '0.7666', '16.1870', '0.0030'], [
    'Pipeline', '0.7588', '27.9367', '0.7325'], [
    'LogisticRegression_unsemp', '0.7541', '22.0845', '0.0024'], [
    'CatBoost', '', '', ''], [
    'RandomForestClassifier', '0.2599', '1.8332', '0.1550'], [
    'DecisionTreeClassifier', '0.6038', '30.3404', '0.0113']

columns = ['type', 'F1', 'learn_time', 'predict_time']
final_models = pd.DataFrame(data=models , columns=columns)
final_models

,type,F1,learn_time,predict_time
0,LogisticRegression,0.7666,16.1870,0.0030
1,Pipeline,0.7588,27.9367,0.7325
2,LogisticRegression_unsemp,0.7541,22.0845,0.0024
3,CatBoost,,,
4,RandomForestClassifier,0.2599,1.8332,0.1550
5,DecisionTreeClassifier,0.6038,30.3404,0.0113


Лучшая модель это логистическая регрессия с несбалансированными классами. Проверим на тестовой выборке

Проверим на тестовой выборке

In [ ]:
target_train_common = data_train_common['toxic']
features_train_common = data_train_common['clear_text']

target_test = data_test['toxic']
features_test = data_test['clear_text']

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train_common) 
tf_idf_test = count_tf_idf.transform(features_test) 
tf_idf_train.shape
model_test = LogisticRegression(C=10.0, random_state = 0, solver = 'liblinear', class_weight= 'balanced')
model_test.fit (tf_idf_train, target_train_common)
print('F1 метрика для логистической регрессии на тестовой выборке', f1_score(target_test, model_test.predict(tf_idf_test)))

print('Время обучения:', learn_time(model_test, tf_idf_train,target_train_common))
print('Время предсказания:', predict_time(model_test, tf_idf_test))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


F1 метрика для логистической регрессии на тестовой выборке 0.7627270020839535
Время обучения: 22.774405479431152
Время предсказания: 0.00784611701965332


## Выводы

Исследовав несколько моделей машинного обучения, среди которых разные типы логистической регрессии (как с дисбалансом классов, так и без), случайный лес, дерево решений,а так же неудачная попытка градиентного бустинга, я пришел к вывоу, что лучшая модель это логистическая регрессия с векторизацией текстов методом TF-IDF/